In [ ]:
import openai
import pandas as pd


In [ ]:
client = openai.Client()


In [ ]:
file = client.files.create(
    file=open("src/samples/sales_data.csv", "rb"),
    purpose="assistants"
)


FileNotFoundError: [Errno 2] No such file or directory: '/src/samples/sales_data.csv'

In [ ]:
print(file.id)


file-HrJ2GUJrmu8udhyAdwkqNG


In [ ]:
assistant = client.beta.assistants.create(
    name="Analista de Dados",
    instructions="Você é um analista que analisa dados sobre vendas",
    tools=[{"type":"code_interpreter"}],
    tool_resources={"code_interpreter":{"file_ids":[file.id]}},
    model="gpt-4o"
)


In [ ]:
# pergunta = "Qual o rating médio das vendas do supermercado"
pergunta = "Gere um gráfico de pizza com o percentual de vendas por linha de produto"


In [ ]:
# Criação da Thread
thread = client.beta.threads.create()
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content=pergunta
)


In [ ]:
# Executa a thread
run = client.beta.threads.runs.create(
    thread_id=thread.id,
    assistant_id=assistant.id,
    instructions="Nome de usuário premium"
)


In [ ]:
# Aguarda a thread rodar
import time
while run.status in ["queued", "in_progress", "cancelling"]:
    time.sleep(1)
    run = client.beta.threads.runs.retrieve(
        thread_id=thread.id,
        run_id=run.id
    )


In [ ]:
# Verifica a resposta
if run.status == "completed":
    mensagens = client.beta.threads.messages.list(
        thread_id=thread.id
    )
    print(mensagens)
else:
    print(f"Erro {run.status}")


SyncCursorPage[Message](data=[Message(id='msg_BIwd18EaJ1Yrh8ozoAkWZ1qI', assistant_id='asst_kEXoysPcMokxwWh4omsP7vSe', attachments=[], completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value='O arquivo parece estar vazio ou não contém dados no formato esperado. Você pode verificar se o arquivo enviado está correto ou tentar abri-lo para garantir que ele contém os dados necessários? Alternativamente, você pode tentar enviar o arquivo novamente.'), type='text')], created_at=1745280493, incomplete_at=None, incomplete_details=None, metadata={}, object='thread.message', role='assistant', run_id='run_JeM7TcLc566UwFP8jE5OpWh4', status=None, thread_id='thread_jpiOok7W6RhpgfnVHjXnYvGX'), Message(id='msg_kGZciHcfNfTcknI2h2Iv3ULk', assistant_id='asst_kEXoysPcMokxwWh4omsP7vSe', attachments=[], completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value='Parece que houve um problema ao tentar ler o arquivo. Isso pode ter ocorrido porque o arquivo não está no fo

In [ ]:
mensagens.data[0].content[0].text.value


'O arquivo parece estar vazio ou não contém dados no formato esperado. Você pode verificar se o arquivo enviado está correto ou tentar abri-lo para garantir que ele contém os dados necessários? Alternativamente, você pode tentar enviar o arquivo novamente.'

In [ ]:
# Analisando os passos do modelo
run_steps = client.beta.threads.runs.steps.list(
    thread_id=thread.id,
    run_id=run.id
)


In [ ]:
for step in run_steps.data[::-1]:
    print(f"\n====Step {step.step_details.type}")
    if step.step_details.type == "tool_calls":
        for tool_call in step.step_details.tool_calls:
            print("=" *10)
            print(tool_call.code_interpreter.input)
            print("="*10)
    if step.step_details.type == "message_creation":
        message = client.beta.threads.messages.retrieve(
            thread_id=thread.id,
            message_id=step.step_details.message_creation.message_id
        )
        if message.content[0].type == "text":
            print(message.content[0].text.value)
        if message.content[0].type == "image_file":
            file_id = message.content[0].image_file.file_id
            image_data = client.files.content(file_id)
            with open(f"files/{file.id}.png", "wb") as f:
                f.write(image_data.read())
                print(f"Imagem {file_id} salva")
        
        # print(message.content[0].text.value)



====Step message_creation
Vamos começar analisando o arquivo que você enviou para entender quais dados ele contém. Depois disso, poderei gerar o gráfico de pizza com o percentual de vendas por linha de produto. Vou carregar e inspecionar os dados agora.

====Step tool_calls
# Importing necessary libraries
import pandas as pd

# Loading the uploaded file to inspect its contents
file_path = '/mnt/data/file-HrJ2GUJrmu8udhyAdwkqNG'
data = pd.read_excel(file_path)
data.head()

====Step message_creation
Parece que houve um problema ao tentar ler o arquivo. Isso pode ter ocorrido porque o arquivo não está no formato Excel. Vou tentar abrir o arquivo como um arquivo CSV para verificar seu conteúdo. Vamos fazer isso agora.

====Step tool_calls
# Attempting to load the file as CSV to inspect its contents
try:
    data = pd.read_csv(file_path)
    data.head()
except Exception as e:
    e

====Step message_creation
O arquivo parece estar vazio ou não contém dados no formato esperado. Você pode ve